In [5]:
import pymorphy2
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
import spacy
import torch  # использую flair
!pip install flair
import flair
import nltk; nltk.download('stopwords')
!python3 -m spacy download en
import re


     |████████████████████████████████| 337kB 2.7MB/s 
     |████████████████████████████████| 798kB 6.0MB/s 
     |████████████████████████████████| 1.3MB 15.5MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 276kB 37.4MB/s 
     |████████████████████████████████| 983kB 48.5MB/s 
     |████████████████████████████████| 1.1MB 49.0MB/s 
     |████████████████████████████████| 2.9MB 36.5MB/s 
     |████████████████████████████████| 890kB 42.2MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116677 sha256=34167186701ba3b745b901389f76dd0dcf0a6ca22cd35873126c9f7afd355d7e
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=11c74d618f833e718b1ccfbd43444579519198e3a9b73603a50a7af2f9aca17d
  Stored in directory: /root/.cache/pip/whee

In [6]:
# сначала поработаю с русским
text_ru = 'К счастью, больной был прооперирован удачно. Новый факультет, образованный в институте по приказу министерства, пользуется большой популярностью. Девушка очень образованна. Больной котенок нашел дом и выздоровел! Морозна ночь, все небо ясно. Он не заметил нас и прошел мимо, хотя вокруг было относительно светло, и мне стало всё ясно. Мой друг снова прошел мимо меня, не поздоровавшись, зато новые друзья были мне рады. Что же ты стоишь? Что ты будешь делать? Он сказал мне, что не приедет. Он ушел, как стемнело. Она обратила внимание на то, как стало темно. У меня в руках была ручная пила. Три дня я пила только воду.'
text_ru = text_ru.lower()  # так будет удобнее сравнивать рез-ты, потому что pymorphy сам приводит всё к нижнему регистру

# чищу и токенизирую для pymorphy и mystem
ru_text_clean = re.sub('[₽#&-;\*\(\(,.!?\d]', ' ', text_ru)
ru_text_split = ru_text_clean.split()

In [7]:
ru_pymorphy_raw = []  # использую pymorphy
morph = pymorphy2.MorphAnalyzer()
for i in range(len(ru_text_split)):
    ru_pymorphy_raw.append(morph.parse(ru_text_split[i]))
ru_pymorphy_str = str(ru_pymorphy_raw)
# привожу к единому стандарту. не стала делать функцию, тк у всех теггеров разные теги…
ru_pymorphy_str = re.sub('ADJ[SF]', 'ADJ', ru_pymorphy_str)
ru_pymorphy_str = re.sub('PRT[SF]', 'VERB', ru_pymorphy_str)
ru_pymorphy_str = re.sub('GRND', 'VERB', ru_pymorphy_str)
ru_pymorphy_str = re.sub('INFN', 'VERB', ru_pymorphy_str)
ru_pymorphy_str = re.sub('ADVB', 'ADV', ru_pymorphy_str)
ru_pymorphy_str = re.sub('PRCL', 'PTLC', ru_pymorphy_str)
ru_pymorphy_str = re.sub('NPRO', 'PRO', ru_pymorphy_str)
ru_pymorphy_str = re.sub('NUMR', 'NUM', ru_pymorphy_str)
ru_pymorphy = re.findall('\[Parse\(word=\'([а-яё]+)\',\stag=OpencorporaTag\(\'([A-Z]+)', ru_pymorphy_str)

In [15]:
from pymystem3 import Mystem
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linu..
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

m = Mystem()  # использую mystem. в колабе что-то тормозит (было написано что-то про установку mystem откуда-то…) 
ru_text_str = str(ru_text_split)  # нашла решение, которое приписала выше, но оно тоже не помогает. 
ru_mystem_raw = m.analyze(ru_text_str)  # в след. ячейке пропишу от руки то, что получила в этом месте кода 
ru_mystem_str = str(ru_mystem_raw)  # + добавлю в реп файл «2.py», где есть вся русская часть кода. эх, а хотела всё красиво собрать в одном месте…
ru_mystem_str = re.sub('S', 'NOUN', ru_mystem_str)  # привожу к единому стандарту
ru_mystem_str = re.sub('A', 'ADJ', ru_mystem_str)
ru_mystem_str = re.sub('PR', 'PREP', ru_mystem_str)
ru_mystem_str = re.sub('V', 'VERB', ru_mystem_str)
ru_mystem_str = re.sub('[AS]PRO', 'PRO', ru_mystem_str)
ru_mystem_str = re.sub('PART', 'PTCL', ru_mystem_str)
ru_mystem = re.findall('text\':\s\'([а-яА-Яё]+)\',\s\'[a-z]+\':\s\[\{\'wt\':\s[\d\.]+,\s\'lex\':\s\'[а-яё]+\',\s\'gr\':\s\'([A-Z]+)', ru_mystem_str)

--2020-10-21 01:35:25--  http://download.cdn.yandex.net/mystem/mystem-3.0-linu..
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.242, 5.45.205.245, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.242|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskmar02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linu.. [following]
--2020-10-21 01:35:26--  http://cache-mskmar02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linu..
Resolving cache-mskmar02.cdn.yandex.net (cache-mskmar02.cdn.yandex.net)... 5.45.222.26, 2a02:6b8:0:2b03::e2
Connecting to cache-mskmar02.cdn.yandex.net (cache-mskmar02.cdn.yandex.net)|5.45.222.26|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-10-21 01:35:27 ERROR 404: Not Found.

tar: mystem-3.0-linux3.1-64bit.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
cp: cannot sta

KeyboardInterrupt: ignored

In [16]:
ru_mystem = [('к', 'PREP'), ('счастью', 'NOUN'), ('больной', 'ADJ'), ('был', 'VERB'), ('прооперирован', 'VERB'), ('удачно', 'ADJDVERB'), ('новый', 'ADJ'), ('факультет', 'NOUN'), ('образованный', 'ADJ'), ('в', 'PREP'), ('институте', 'NOUN'), ('по', 'PREP'), ('приказу', 'NOUN'), ('министерства', 'NOUN'), ('пользуется', 'VERB'), ('большой', 'ADJ'), ('популярностью', 'NOUN'), ('девушка', 'NOUN'), ('очень', 'ADJDVERB'), ('образованна', 'ADJ'), ('больной', 'ADJ'), ('котенок', 'NOUN'), ('нашел', 'VERB'), ('дом', 'NOUN'), ('и', 'CONJ'), ('выздоровел', 'VERB'), ('морозна', 'ADJ'), ('ночь', 'NOUN'), ('все', 'NOUNPREPO'), ('небо', 'NOUN'), ('ясно', 'ADJDVERB'), ('он', 'NOUNPREPO'), ('не', 'PADJRT'), ('заметил', 'VERB'), ('нас', 'NOUNPREPO'), ('и', 'CONJ'), ('прошел', 'VERB'), ('мимо', 'ADJDVERB'), ('хотя', 'CONJ'), ('вокруг', 'ADJDVERB'), ('было', 'VERB'), ('относительно', 'ADJDVERB'), ('светло', 'ADJDVERB'), ('и', 'CONJ'), ('мне', 'NOUNPREPO'), ('стало', 'VERB'), ('всё', 'ADJDVERB'), ('ясно', 'ADJDVERB'), ('мой', 'ADJPREPO'), ('друг', 'NOUN'), ('снова', 'ADJDVERB'), ('прошел', 'VERB'), ('мимо', 'PREP'), ('меня', 'NOUNPREPO'), ('не', 'PADJRT'), ('поздоровавшись', 'VERB'), ('зато', 'CONJ'), ('новые', 'ADJ'), ('друзья', 'NOUN'), ('были', 'VERB'), ('мне', 'NOUNPREPO'), ('рады', 'ADJ'), ('что', 'NOUNPREPO'), ('же', 'PADJRT'), ('ты', 'NOUNPREPO'), ('стоишь', 'VERB'), ('что', 'CONJ'), ('ты', 'NOUNPREPO'), ('будешь', 'VERB'), ('делать', 'VERB'), ('он', 'NOUNPREPO'), ('сказал', 'VERB'), ('мне', 'NOUNPREPO'), ('что', 'CONJ'), ('не', 'PADJRT'), ('приедет', 'VERB'), ('он', 'NOUNPREPO'), ('ушел', 'VERB'), ('как', 'CONJ'), ('стемнело', 'VERB'), ('она', 'NOUNPREPO'), ('обратила', 'VERB'), ('внимание', 'NOUN'), ('на', 'PREP'), ('то', 'NOUNPREPO'), ('как', 'CONJ'), ('стало', 'VERB'), ('темно', 'ADJDVERB'), ('у', 'PREP'), ('меня', 'NOUNPREPO'), ('в', 'PREP'), ('руках', 'NOUN'), ('была', 'VERB'), ('ручная', 'ADJ'), ('пила', 'NOUN'), ('три', 'NUM'), ('дня', 'NOUN'), ('я', 'NOUNPREPO'), ('пила', 'VERB'), ('только', 'PADJRT'), ('воду', 'NOUN')]

In [11]:
segmenter = Segmenter()  # использую natasha
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(text_ru)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
ru_natasha_raw = doc.tokens
ru_natasha_str = str(ru_natasha_raw)
ru_natasha_str = re.sub('ADP', 'PREP', ru_natasha_str)  # привожу к единому стандарту
ru_natasha_str = re.sub('AUX', 'VERB', ru_natasha_str)
ru_natasha_str = re.sub('CCONJ', 'CONJ', ru_natasha_str)
ru_natasha_str = re.sub('SCONJ', 'CONJ', ru_natasha_str)
ru_natasha_str = re.sub('PRON', 'PRO', ru_natasha_str)
ru_natasha_str = re.sub('PART', 'PTCL', ru_natasha_str)
ru_natasha = re.findall('text=\'([а-яА-Яё]+)\',\spos=\'([A-Z]+)', ru_natasha_str)

In [12]:
#теперь ручная разметка. разберемся сразу с обоими текстами:
ru_manually_raw = 'к PREP счастью NOUN больной NOUN был VERB прооперирован VERB удачно ADV новый ADJ факультет NOUN образованный VERB в PREP институте NOUN по PREP приказу NOUN министерства NOUN пользуется VERB большой ADJ популярностью NOUN девушка NOUN очень ADV образованная ADJ больной ADJ котенок NOUN нашел VERB дом NOUN и CONJ выздоровел VERB морозна ADJ ночь NOUN все ADJ небо NOUN ясно ADJ он PRO не PTCL заметил VERB нас PRO и CONJ прошел VERB мимо ADV хотя CONJ вокруг PREP было VERB относительно ADV светло ADV и CONJ мне PRO стало VERB всё NOUN ясно ADV мой PRO друг NOUN снова ADV прошел VERB мимо PREP меня PRO не PTCL поздоровавшись VERB зато CONJ новые ADJ друзья NOUN были VERB мне PRO рады ADJ что ADV же PTCL ты PRO стоишь VERB что PRO ты PRO будешь VERB делать VERB он PRO сказал VERB мне PRO что CONJ не PTCL приедет VERB он PRO ушел VERB как CONJ стемнело VERB она PRO обратила VERB внимание NOUN на PREP то PRO как PRO стало VERB темно ADV у PREP меня PRO в PREP руках NOUN была VERB ручная ADJ пила NOUN три NUM дня NOUN я PRO пила VERB только ADV воду NOUN'
en_manually_raw = 'Back VERB up ADV and CONJ give VERB me PRO an ART answer NOUN Stop VERB and CONJ answer VERB my PRO question NOUN He PRO wants VERB to PREP frame VERB me PRO for PREP stealing VERB the ART gold ADJ frame NOUN If CONJ you PRO set VERB us PRO on PREP fire NOUN the ART boss NOUN will VERB definitely ADV fire VERB you PRO I PRO saw VERB a ART giant ADJ saw NOUN outside ADV the ART shelter NOUN it PRO destroyed VERB the ART tree NOUN that CONJ was VERB about PREP to PREP flower VERB Why ADV did VERB you PRO fool VERB me PRO into PREP believing VERB that CONJ this ART function NOUN would VERB work VERB Could VERB you PRO please ADV iron VERB my PRO suit NOUN Did VERB I PRO tell VERB you PRO that CONJ they PRO paid VERB me PRO a ART visit NOUN the ART other ADJ night NOUN when ADV I PRO was VERB trying VERB to PREP finally ADV get VERB some ADJ sleep NOUN Email VERB me PRO later ADV and CONJ we PRO will VERB discuss VERB that ART email NOUN Can VERB you PRO guess VERB what ADJ type NOUN of PREP glue NOUN I PRO used VERB to PREP do VERB this ART object NOUN Wake VERB up ADV and CONJ help VERB me PRO to PREP find VERB my PRO watch NOUN'


def manually_into_list(string_raw):  # преобразовываю ручную разметку в годный для сравненеия вид
    string_raw = string_raw.split()
    list = []
    for k in range(len(string_raw)):
        if (k != 0 and k % 2 == 1):
            pass
        else:
            list.append((string_raw[k], string_raw[k + 1]))
    return list


ru_manually = manually_into_list(ru_manually_raw)
en_manually = manually_into_list(en_manually_raw)

In [17]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return len(lst3)


def accuracy_check(list1, list2):  # функция вычисления точности
    accuracy = round((intersection(list1, list2)/101), 2)
    return accuracy


pymorphy_accuracy = accuracy_check(ru_pymorphy, ru_manually)  # считаю
mystem_accuracy = accuracy_check(ru_mystem, ru_manually)
natasha_accuracy = accuracy_check(ru_natasha, ru_manually)

if pymorphy_accuracy > mystem_accuracy:  # находит и объявляет самый точный русский теггер
    if pymorphy_accuracy > natasha_accuracy:
        print('pymorphy самый точный из русских:', pymorphy_accuracy, ' vs ', mystem_accuracy, ' и ', natasha_accuracy)
    else:
        print('natasha самая точная из русских:', natasha_accuracy, ' vs ', pymorphy_accuracy, ' и ', mystem_accuracy)
elif pymorphy_accuracy < mystem_accuracy:
    if mystem_accuracy > natasha_accuracy:
        print('mystem самый точный из русских:', mystem_accuracy, ' vs ', pymorphy_accuracy, ' и ', natasha_accuracy)
    else:
        print('natasha самая точная из русских:', natasha_accuracy, ' vs ', pymorphy_accuracy, ' и ', mystem_accuracy)
else:
    if pymorphy_accuracy > natasha_accuracy:
        print('pymorphy и mystem самые точные из русских:', pymorphy_accuracy, ' и ', mystem_accuracy, ' vs ', natasha_accuracy)
    else:
        print('natasha самая точная из русских:', natasha_accuracy, ' vs ', pymorphy_accuracy, ' и ', mystem_accuracy)


natasha самая точная из русских: 0.91  vs  0.84  и  0.65


In [29]:
# перехожу к английскому
text_en = 'Back up and give me an answer. Stop and answer my question. He wants to frame me for stealing the gold frame. If you set us on fire the boss will definitely fire you. I saw a giant saw outside the shelter, it destroyed the tree that was about to flower. Why did you fool me into believing that this function would work? Could you please iron my suit? Did I tell you that they paid me a visit the other night when I was trying to finally get some sleep? Email me later and we will discuss that email. Can you guess what type of glue I used to do this object? Wake up and help me to find my watch!'


def standart(text_str):  # привожу к единому стандарту. в этот раз функция, потому что все 3 теггера используют одинаковые тэги (отличные от моих)
    text_str = re.sub('RB', 'ADV', text_str)
    text_str = re.sub('CC', 'CONJ', text_str)
    text_str = re.sub('VB[ZGPD]', 'ADV', text_str)
    text_str = re.sub('VB', 'VERB', text_str)
    text_str = re.sub('MD', 'VERB', text_str)
    text_str = re.sub('PRP', 'PRO', text_str)
    text_str = re.sub('PRP\$', 'PRO', text_str)
    text_str = re.sub('DT', 'ART', text_str)
    text_str = re.sub('NN', 'NOUN', text_str)
    text_str = re.sub('TO', 'PREP', text_str)
    text_str = re.sub('IN', 'PREP', text_str)
    text_str = re.sub('WDT', 'CONJ', text_str)
    text_str = re.sub('JJ', 'ADJ', text_str)
    text_str = re.sub('WRB', 'ADV', text_str)
    text_str = re.sub('NNP', 'PRO', text_str)
    return text_str

In [20]:
nlp = spacy.load("en_core_web_sm")  # начинаю со spacy

en_text_clean = re.sub('[₽#&-;\*\(\(,.!?\d]', '', text_en)
doc = nlp(en_text_clean)
en_spacy_list = []
en_spacy_str = ''
for token in doc:  # проще было бы сделать сразу список списков, но нужно будет менять теги, чтоб совпадали с моими, поэтому нужна str (для re.sub)
    en_spacy_str += token.text
    en_spacy_str += ' '
    en_spacy_str += token.tag_
    en_spacy_str += ' '

en_spacy_str = standart(en_spacy_str)
en_spacy = re.findall('([a-zA-Z]+)\s([A-Z]+)', en_spacy_str)

In [40]:
from tqdm import tqdm
from flair.data import Sentence  # использую flair
from flair.models import SequenceTagger
nltk.download('punkt')

pos = SequenceTagger.load('pos-fast')

en_flair_raw = []
en_flair_list = nltk.tokenize.sent_tokenize(text_en)

for i in tqdm(en_flair_list):
  sentence = Sentence(i)
  pos.predict(sentence)
  en_flair_raw.append(sentence.to_tagged_string())

import re
en_flair_str = str(en_flair_raw)
en_flair_str = standart(en_flair_str)
en_flair = re.findall('([a-zA-Z]+)\s<([A-Z]+)', en_flair_str)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2020-10-21 01:58:12,837 loading file /root/.flair/models/en-pos-ontonotes-fast-v0.5.pt


100%|██████████| 11/11 [00:01<00:00,  8.57it/s]


In [34]:
nltk.download('tagsets')  # использую nltk
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
text = word_tokenize(text_en)
en_nltk_raw = nltk.pos_tag(text)
en_nltk_str = str(en_nltk_raw)
en_nltk_str = standart(en_nltk_str)
en_nltk = re.findall('\'([a-zA-Z]+)\',\s\'([A-Z]+)', en_nltk_str)

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[('Back', 'NOUNP'), ('up', 'ADV'), ('and', 'CONJ'), ('give', 'VERB'), ('me', 'PRO'), ('an', 'ART'), ('answer', 'NOUN'), ('Stop', 'NOUNP'), ('and', 'CONJ'), ('answer', 'VERB'), ('my', 'PRO'), ('question', 'NOUN'), ('He', 'PRO'), ('wants', 'ADV'), ('to', 'PREP'), ('frame', 'VERB'), ('me', 'PRO'), ('for', 'PREP'), ('stealing', 'ADV'), ('the', 'ART'), ('gold', 'NOUN'), ('frame', 'NOUN'), ('If', 'PREP'), ('you', 'PRO'), ('set', 'ADV'), ('us', 'PRO'), ('on', 'PREP'), ('fire', 'NOUN'), ('the', 'ART'), ('boss', 'NOUN'), ('will', 'VERB'), ('definitely', 'ADV'), ('fire', 'VERB'), ('you', '

In [42]:
spacy_accuracy = accuracy_check(en_spacy, en_manually)  # считаю
flair_accuracy = accuracy_check(en_flair, en_manually)
nltk_accuracy = accuracy_check(en_nltk, en_manually)

if spacy_accuracy > flair_accuracy:  # находит и объявляет самый точный английский теггер
    if spacy_accuracy > nltk_accuracy:
        print('spacy самый точный из английских:', spacy_accuracy, ' vs ', flair_accuracy, ' и ', nltk_accuracy)
    else:
        print('nltk самая точная из английских:', nltk_accuracy, ' vs ', spacy_accuracy, ' и ', flair_accuracy)
elif spacy_accuracy < flair_accuracy:
    if flair_accuracy > nltk_accuracy:
        print('flair самый точный из английских:', flair_accuracy, ' vs ', spacy_accuracy, ' и ', nltk_accuracy)
    else:
        print('nltk самая точная из английских:', nltk_accuracy, ' vs ', spacy_accuracy, ' и ', flair_accuracy)
else:
    if spacy_accuracy > nltk_accuracy:
        print('spacy и flair самые точные из английских:', spacy_accuracy, ' и ', flair_accuracy, ' vs ', nltk_accuracy)
    else:
        print('nltk самая точная из английских:', nltk_accuracy, ' vs ', spacy_accuracy, ' и ', flair_accuracy)

flair самый точный из английских: 0.92  vs  0.9  и  0.91
